In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI
from dotenv import load_dotenv
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""
    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None
    #This created a function that AI agent could use
    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)
        # Select a random destination
        destination = random.choice(available_destinations)
        # Update the last destination
        self.last_destination = destination
        return destination

In [ ]:
class FoodsPlugin:
    """Suggests famous foods based on destination."""
    @kernel_function(description="Suggests a famous food based on destination.")
    def get_food_suggestion(self) -> Annotated[str, "Returns a food suggestion."]:
        foods = [
            "Pizza in Italy",
            "Sushi in Japan",
            "Paella in Spain",
            "Tacos in Mexico",
            "Croissant in France",
        ]
        return random.choice(foods)

In [ ]:
class PackingPlugin:
    """Suggests travel packing essentials."""
    @kernel_function(description="Suggests essential travel packing items.")
    def get_packing_list(self) -> Annotated[str, "Returns a list of essential packing items."]:
        return "Don't forget your passport, sunglasses, comfy shoes, power bank, and travel pillow!"

In [ ]:
class ItineraryPlugin:
    """Creates a simple travel itinerary for a day."""
    @kernel_function(description="Creates a simple travel itinerary for a day.")
    def create_day_itinerary(self) -> Annotated[str, "Returns a simple plan for a day."]:
        return (
            "Morning: Visit local museum.\n"
            "Afternoon: Lunch at a famous cafe.\n"
            "Evening: Walk by the river and dinner at a seafood restaurant."
        )

In [ ]:
class MapsPlugin:
    """Generates Google Maps links for destinations."""
    @kernel_function(description="Generates a Google Maps link for a destination.")
    def get_google_maps_link(self) -> Annotated[str, "Returns a Google Maps search link."]:
        destinations = [
            "Paris",
            "Tokyo",
            "New York",
            "Barcelona",
            "Sydney",
        ]
        place = random.choice(destinations)
        link = f"https://www.google.com/maps/search/{place.replace(' ', '+')}"
        return f"Here's the map for {place}: {link}"

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)
# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [58]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin(), FoodsPlugin(), PackingPlugin(), ItineraryPlugin(),MapsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI agent that helps plan vacations including destination, food, packing, and maps",
)

In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None
    user_inputs = [
        "Plan me a weekend trip. Include Adventures, road trips, and fun",
    ]
    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()
    # Clean up the thread
    await thread.delete() if thread else None
await main()

# User: Plan me a weekend trip. Include Adventures, road trips, and fun

# TravelAgent: ### Weekend Trip to Sydney, Australia

**Adventure and Fun Itinerary:**

- **Morning:** Start your day with a visit to a local museum to immerse yourself in the culture and history of Sydney.
- **Afternoon:** Enjoy a delicious lunch at a famous café, where you can indulge in some local flavors.
- **Evening:** Take a relaxing walk by the river, soaking in the beautiful views, followed by dinner at a seafood restaurant to experience Sydney’s famous fresh catch.

**Food Suggestion:**
While you're in Sydney, make sure to try some authentic **Pizza**, reminiscent of the delightful pizzas you would find in Italy!

**Packing Essentials:**
- Passport
- Sunglasses
- Comfy shoes
- Power bank
- Travel pillow

**Transportation:**
Here’s the link to the map for navigating Sydney: [Google Maps - Sydney](https://www.google.com/maps/search/Sydney)

Enjoy your weekend adventure! If you need further details or adjust